In [1]:
import torch
import torch.nn as nn
import torch.optim as optimizer
import torchvision.transforms as transforms

from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
import os
import sys
import scipy.io as scio

from sklearn.model_selection import train_test_split

In [3]:
image_path = "../image"

classes = []
classCount = 20

for root, dirs, classNames in os.walk(image_path):
	dirs.sort()
	i = 0
	for dir_ in dirs:
		if i < classCount:
			i += 1
			classes.append(os.path.basename(dir_))
		else:
			break

x = list()
y_label = list()
index = 0

for className in classes:
	class_path = os.path.join(data_path, className)
	for root, _, fileNames in os.walk(class_path):
		for fileName in fileNames:
			if fileName != '.DS_Store':
				dataItem = cv.imread(os.path.join(root, fileName), cv.IMREAD_GRAYSCALE)
				transf = transforms.ToTensor()
				imageTensor = transf(dataItem)
				if imageTensor.size()[2] != 148:
					print(fileName)
				x.append(imageTensor)
				y_label.append([index, int(root[-4:])])
	index += 1

y_label = np.array(y_label)
y = y_label[:, 0]

print(y)
print(len(x), len(y))
print(x[0].shape)


[ 0  0  0 ... 19 19 19]
7032 7032
torch.Size([1, 369, 148])


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8)

print("x_train:", len(x_train))
print("x_test:", len(x_test))
print("Shape of x_train:", x_train[0].shape)
print("Shape of y_train:", np.shape(y_train))
print("Shape of x_test:", x_test[0].shape)
print("Shape of y_test:", np.shape(y_test))


x_train: 1406
x_test: 5626
Shape of x_train: torch.Size([1, 369, 148])
Shape of y_train: (1406,)
Shape of x_test: torch.Size([1, 369, 148])
Shape of y_test: (5626,)


In [5]:
x_train = torch.stack(x_train, dim=0)
x_test = torch.stack(x_test, dim=0)
print(x_train.size())
print(x_test.size())

torch.Size([1406, 1, 369, 148])
torch.Size([5626, 1, 369, 148])


In [6]:
batch_size = 20

class TrainDatasets(Dataset):
	def __init__(self):
		self.len = x_train.shape[0]
		self.X_train = x_train
		self.Y_train = torch.from_numpy(y_train).long()

	def __getitem__(self, index):
		return self.X_train[index], self.Y_train[index]

	def __len__(self):
		return self.len

class TestDatasets(Dataset):
	def __init__(self):
		self.len = x_test.shape[0]
		self.X_test = x_test
		self.Y_test = torch.from_numpy(y_test).long()

	def __getitem__(self, index):
		return self.X_test[index], self.Y_test[index]

	def __len__(self):
		return self.len

train_dataset = TrainDatasets()
test_dataset = TestDatasets()

train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
print(classCount)

20


In [8]:
_in_channel = 1

class Flatten(nn.Module):
	def forward(self, x):
		batch_size = x.shape[0]
		return x.view(batch_size, -1)

class SpecCNN(nn.Module):
	def __init__(self, in_channel=_in_channel):
		super(SpecCNN, self).__init__()
		self.cnn = nn.Sequential(
			nn.Conv2d(1, 128, 50, stride=3),
			nn.ReLU(),
			nn.BatchNorm2d(128),
			nn.MaxPool2d(kernel_size=5, stride=3),
			nn.Dropout(p=0.1),

			nn.Conv2d(128, 32, 3, stride=2),
			nn.ReLU(),
			nn.BatchNorm2d(32),
			nn.MaxPool2d(kernel_size=2, stride=2),
			nn.Dropout(p=0.1),

			Flatten(),
			nn.Linear(in_features=512, out_features=256),
			nn.ReLU(),
			nn.Dropout(p=0.1),
			nn.Linear(in_features=256, out_features=classCount)
		)

	def forward(self, x, ex_features=None):
		return self.cnn(x)


In [9]:
model = SpecCNN()
print(model)

SpecCNN(
  (cnn): Sequential(
    (0): Conv2d(1, 128, kernel_size=(50, 50), stride=(3, 3))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=5, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.1, inplace=False)
    (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(2, 2))
    (6): ReLU()
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.1, inplace=False)
    (10): Flatten()
    (11): Linear(in_features=512, out_features=256, bias=True)
    (12): ReLU()
    (13): Dropout(p=0.1, inplace=False)
    (14): Linear(in_features=256, out_features=20, bias=True)
  )
)


In [10]:
Loss = nn.CrossEntropyLoss()
optimi = optimizer.Adam(model.parameters(), lr=0.0004)

In [11]:
correct_list = []

def train(epoch):
	running_loss = 0.0
	for batch_idx, data in enumerate(train_loader):
		inputs, target = data
		optimi.zero_grad()
		outputs = model(inputs)
		loss = Loss(outputs, target)
		loss.backward()
		optimi.step()
		running_loss += loss.item()
		if batch_idx % 10 == 9:
			print('[%d, %5d] Loss: %.8f' % (epoch + 1, batch_idx + 1, running_loss / 300))
			running_loss = 0.0

def test():
	correct = 0
	total = 0
	with torch.no_grad():
		for data in test_loader:
			images, labels = data
			outputs = model(images)
			_, predicted = torch.max(outputs.data, dim=1)
			total += len(labels)
			correct += (predicted == labels).sum().item()

		correct_list.append((100 * correct) / total)
		print('Accuracy on test set: %d %%' % (100 * correct / total))

for epoch in range(20):
	train(epoch)
	test()


[1,    10] Loss: 0.09836342
[1,    20] Loss: 0.09784160
[1,    30] Loss: 0.09280744
[1,    40] Loss: 0.08603022
[1,    50] Loss: 0.08132563
[1,    60] Loss: 0.07601061
[1,    70] Loss: 0.07267378
Accuracy on test set: 40 %
[2,    10] Loss: 0.06336585
[2,    20] Loss: 0.05587473
[2,    30] Loss: 0.05389811
[2,    40] Loss: 0.04754989
[2,    50] Loss: 0.04721261
[2,    60] Loss: 0.04076739
[2,    70] Loss: 0.03611719
Accuracy on test set: 67 %
[3,    10] Loss: 0.03136066
[3,    20] Loss: 0.03040554
[3,    30] Loss: 0.02653314
[3,    40] Loss: 0.02501999
[3,    50] Loss: 0.02922982
[3,    60] Loss: 0.02525796
[3,    70] Loss: 0.02331463
Accuracy on test set: 75 %
[4,    10] Loss: 0.02311069
[4,    20] Loss: 0.01908012
[4,    30] Loss: 0.01972912
[4,    40] Loss: 0.02262194
[4,    50] Loss: 0.01986765
[4,    60] Loss: 0.01864532
[4,    70] Loss: 0.02139099
Accuracy on test set: 78 %
[5,    10] Loss: 0.01265982
[5,    20] Loss: 0.01356320
[5,    30] Loss: 0.01384961
[5,    40] Loss: 0.01566

In [16]:
import time

model_path = '../model'

localtime = time.asctime(time.localtime(time.time()))
pth_name = str(localtime) + '.pth'

torch.save(model, os.path.join(model_path, pth_name))